In [1]:
import pandas as pd
import numpy as np
from keras import Sequential, layers
from keras.optimizers import Adam
from keras.datasets import cifar10
from keras.layers import Conv2D,LeakyReLU,BatchNormalization,Flatten,Dense,Reshape,Conv2DTranspose,Input,Embedding,LSTM
import matplotlib.pyplot as plt
from keras import initializers
from keras.models import Model
from keras.utils import np_utils
from keras import backend as K
from keras.utils import to_categorical
import matplotlib.pyplot as plt

Using TensorFlow backend.


### Define functions for reading and preprocessing

In [0]:
def convert_to_array(file):
    res = list()
    #Open file and read add every value from a line in a list
    with open(file,'r') as f:
        for line in f:
            res.append(float(line))
    res = np.asarray(res)        
    f.close()
    return res

In [0]:
def create_dataset(dataset, prev):
	dataX, dataY = [], []
	for i in range(len(dataset)-prev-1):
		a = dataset[i:(i+prev), 0]
		dataX.append(a)
		dataY.append(dataset[i + prev, 0])
		print(str(a) + ' s '+str(dataset[i + prev, 0]))
	return np.array(dataX), np.array(dataY)

## Read PM2.5 values for training

In [0]:
X09 = convert_to_array('aqi_09_2017')
X10 = convert_to_array('aqi_10_2017')
X11 = convert_to_array('aqi_11_2017')
X12 = convert_to_array('aqi_12_2017')
data = np.concatenate([X09, X10, X11, X12], axis = 0)
data = data.reshape(data.shape[0],1)
train_size = int(len(data) * 0.67)
test_size = len(data) - train_size
x_train, x_test = data[0:train_size,:], data[train_size:len(data),:]

In [0]:
x_train.shape

(1961, 1)

## Reshape to feed into LSTM

In [8]:
look_back = 1
#By doing this we are making the next PM2.5 value as the target for current
#Shifting the values by 1.
x_train, y_train = create_dataset(x_train, look_back)
x_test, y_test = create_dataset(x_test, look_back)

[13.9] s 14.4
[14.4] s 16.7
[16.7] s 17.3
[17.3] s 17.6
[17.6] s 15.5
[15.5] s 14.6
[14.6] s 17.1
[17.1] s 16.6
[16.6] s 17.5
[17.5] s 11.7
[11.7] s 9.2
[9.2] s 6.4
[6.4] s 6.1
[6.1] s 6.2
[6.2] s 6.9
[6.9] s 7.4
[7.4] s 8.0
[8.] s 7.8
[7.8] s 8.9
[8.9] s 7.5
[7.5] s 6.3
[6.3] s 7.1
[7.1] s 6.0
[6.] s 7.0
[7.] s 9.4
[9.4] s 9.6
[9.6] s 10.9
[10.9] s 14.1
[14.1] s 10.8
[10.8] s 11.8
[11.8] s 12.4
[12.4] s 16.1
[16.1] s 16.7
[16.7] s 12.1
[12.1] s 9.1
[9.1] s 7.3
[7.3] s 8.4
[8.4] s 7.2
[7.2] s 6.4
[6.4] s 7.3
[7.3] s 5.5
[5.5] s 5.6
[5.6] s 5.9
[5.9] s 5.8
[5.8] s 5.8
[5.8] s 5.8
[5.8] s 6.4
[6.4] s 5.5
[5.5] s 7.2
[7.2] s 9.9
[9.9] s 11.2
[11.2] s 11.1
[11.1] s 9.1
[9.1] s 11.6
[11.6] s 9.0
[9.] s 10.9
[10.9] s 10.6
[10.6] s 8.5
[8.5] s 7.7
[7.7] s 7.6
[7.6] s 8.4
[8.4] s 8.2
[8.2] s 9.3
[9.3] s 8.9
[8.9] s 8.1
[8.1] s 8.0
[8.] s 7.2
[7.2] s 8.6
[8.6] s 9.3
[9.3] s 8.1
[8.1] s 8.6
[8.6] s 8.2
[8.2] s 8.6
[8.6] s 8.2
[8.2] s 8.4
[8.4] s 10.0
[10.] s 10.6
[10.6] s 10.5
[10.5] s 10.6
[10.

## Reshape training and testing datashape

In [0]:
#Since the LSTM takes as input (samples,timestemps,features)
x_train = np.reshape(x_train, (x_train.shape[0], 1, x_train.shape[1]))
x_test = np.reshape(x_test, (x_test.shape[0], 1, x_test.shape[1]))

## Define model 

In [12]:

model = Sequential()
model.add(LSTM(128, input_shape=(1,1),return_sequences=True))
model.add(LSTM(128))
model.add(Dense(1))
model.compile(loss='mean_absolute_error', optimizer='adam',metrics=['accuracy'])


## Start training

In [13]:
model.fit(x_train, y_train, nb_epoch=1000, batch_size=64, verbose=2)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/1000





 - 5s - loss: 19.6805 - acc: 0.0000e+00
Epoch 2/1000
 - 0s - loss: 10.7368 - acc: 0.0260
Epoch 3/1000
 - 0s - loss: 7.6664 - acc: 0.0587
Epoch 4/1000
 - 0s - loss: 6.1602 - acc: 0.0929
Epoch 5/1000
 - 0s - loss: 5.2109 - acc: 0.0934
Epoch 6/1000
 - 0s - loss: 4.5462 - acc: 0.1092
Epoch 7/1000
 - 0s - loss: 4.1062 - acc: 0.1041
Epoch 8/1000
 - 0s - loss: 3.7104 - acc: 0.1113
Epoch 9/1000
 - 0s - loss: 3.4328 - acc: 0.1087
Epoch 10/1000
 - 0s - loss: 3.1990 - acc: 0.1123
Epoch 11/1000
 - 0s - loss: 3.0317 - acc: 0.1184
Epoch 12/1000
 - 0s - loss: 2.9047 - acc: 0.1123
Epoch 13/1000
 - 0s - loss: 2.8322 - acc: 0.1108
Epoch 14/1000
 - 0s - loss: 2.6961 - acc: 0.1184
Epoch 15/1000
 - 0s - loss: 2.6286 - acc: 0.1266
Epoch 16/1000
 - 0s - loss: 2.5704 - acc: 0.1220
Epoch 17/1000
 - 0s - loss: 2.5118 - acc: 0.1215
Epoch 18/1000
 - 0s - loss: 2.4791 - acc: 0.1210
Epoch 19/1000
 - 0s

### Evaluate model

In [14]:
model.evaluate(x_test,y_test)

965/965 [==============================] - 0s 268us/step


[3.7770617922353003, 0.16580310880829016]

### See predicted values

In [15]:
y = model.predict(x_test)

y = np.round(y,0)
for i in range(len(y)):
  print(str(y[i]) + ' '+ str(y_test[i]))

[25.] 29.0
[29.] 34.0
[34.] 36.0
[36.] 40.0
[40.] 40.0
[40.] 38.0
[38.] 43.0
[43.] 41.0
[41.] 39.0
[39.] 38.0
[38.] 41.0
[41.] 39.0
[39.] 37.0
[37.] 44.0
[43.] 48.0
[47.] 52.0
[51.] 52.0
[51.] 55.0
[53.] 56.0
[54.] 53.0
[52.] 47.0
[46.] 39.0
[39.] 34.0
[34.] 41.0
[41.] 47.0
[46.] 52.0
[51.] 54.0
[53.] 55.0
[53.] 51.0
[50.] 29.0
[29.] 14.0
[14.] 12.0
[12.] 10.0
[10.] 8.0
[8.] 8.0
[8.] 8.0
[8.] 11.0
[11.] 15.0
[15.] 17.0
[17.] 15.0
[15.] 11.0
[11.] 11.0
[11.] 13.0
[13.] 10.0
[10.] 9.0
[9.] 8.0
[8.] 9.0
[9.] 10.0
[10.] 10.0
[10.] 12.0
[12.] 15.0
[15.] 19.0
[19.] 19.0
[19.] 21.0
[21.] 22.0
[22.] 18.0
[18.] 17.0
[17.] 19.0
[19.] 15.0
[15.] 13.0
[13.] 13.0
[13.] 14.0
[14.] 21.0
[21.] 27.0
[27.] 25.0
[25.] 19.0
[19.] 19.0
[19.] 17.0
[17.] 16.0
[16.] 12.0
[12.] 11.0
[11.] 10.0
[10.] 11.0
[11.] 13.0
[13.] 17.0
[17.] 19.0
[19.] 22.0
[22.] 21.0
[21.] 22.0
[22.] 26.0
[26.] 28.0
[28.] 24.0
[24.] 24.0
[24.] 24.0
[24.] 24.0
[24.] 25.0
[25.] 29.0
[29.] 32.0
[32.] 38.0
[38.] 35.0
[35.] 34.0
[34.] 32.0
